In [1]:
import pandas as pd
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [2]:
# df.info()
# df.describe()
df.describe()
# df.describe(include=object)

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [3]:
tm = df.groupby("Survived").mean()
tm
v1 = tm.iloc[0,:]
v2 = tm.iloc[1,:]
abs((v2-v1)/(v1+v2))

PassengerId    0.002971
Pclass         0.129755
Age            0.038706
SibSp          0.077914
Parch          0.170176
Fare           0.372661
dtype: float64

In [4]:
print(df["Name"].head())

def get_title(name):
    if '.' in name:
        return name.split(',')[1].split('.')[0].strip()
    else:
        return 'Unknown'
def title_map(title):
    if title in ['Mr']:
        return 1
    elif title in ['Master']:
        return 3
    elif title in ['Ms','Mlle','Miss']:
        return 4
    elif title in ['Mme','Mrs']:
        return 5
    else:
        return 2
    
df['title'] = df['Name'].apply(get_title).apply(title_map)   

df = df.drop(["PassengerId", "Name", "Ticket"], axis="columns")
df.info()

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
Survived    891 non-null int64
Pclass      891 non-null int64
Sex         891 non-null object
Age         714 non-null float64
SibSp       891 non-null int64
Parch       891 non-null int64
Fare        891 non-null float64
Cabin       204 non-null object
Embarked    889 non-null object
title       891 non-null int64
dtypes: float64(2), int64(5), object(3)
memory usage: 69.7+ KB


In [5]:
!python -m pip install xgboost

You are using pip version 9.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [6]:
df["Sex"] = df["Sex"].replace(["male", "female"], [0, 1])
df["Cabin"] = df["Cabin"].isna()
df = pd.get_dummies(df)
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Sex           891 non-null int64
Age           714 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Cabin         891 non-null bool
title         891 non-null int64
Embarked_C    891 non-null uint8
Embarked_Q    891 non-null uint8
Embarked_S    891 non-null uint8
dtypes: bool(1), float64(2), int64(6), uint8(3)
memory usage: 59.2 KB


In [7]:
tm = df.groupby("Survived").mean()
tm
v1 = tm.iloc[0,:]
v2 = tm.iloc[1,:]
abs((v2-v1)/(v1+v2))

Pclass        0.129755
Sex           0.643977
Age           0.038706
SibSp         0.077914
Parch         0.170176
Fare          0.372661
Cabin         0.185190
title         0.374197
Embarked_C    0.331221
Embarked_Q    0.012168
Embarked_S    0.101449
dtype: float64

In [8]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split


In [9]:
# print(df.groupby("title").Age.mean())
# df["Age"][df["title"]==3] = 7
# mage = df.Age.mean()
# df.Age = df.Age.fillna(mage)
# df = df.drop(["title"], axis="columns")

df["Age"][df["Age"].isna()] = df["Age"].mean()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
Survived      891 non-null int64
Pclass        891 non-null int64
Sex           891 non-null int64
Age           891 non-null float64
SibSp         891 non-null int64
Parch         891 non-null int64
Fare          891 non-null float64
Cabin         891 non-null bool
title         891 non-null int64
Embarked_C    891 non-null uint8
Embarked_Q    891 non-null uint8
Embarked_S    891 non-null uint8
dtypes: bool(1), float64(2), int64(6), uint8(3)
memory usage: 59.2 KB


/home/gaurav/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [10]:
mf = df['Fare'].mean()
df['Fare'] = df['Fare']>mf

# df.loc[ df['Fare'] <= 100, 'Fare'] = 0
# df.loc[ df['Fare'] > 100, 'Fare'] = 3

# # convert from float to int
df['Fare'] = df['Fare'].astype(int)


y = df["Survived"]
df.drop("Survived", axis="columns", inplace=True)
X = df


In [11]:
# model = LogisticRegression()
# model = RandomForestClassifier()

model = XGBClassifier()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=13)
model.fit(X_train, y_train)

print(model.score(X_test,y_test))

from sklearn.metrics import confusion_matrix

yp = model.predict(X_test)
print("Sur", sum(yp!=0))
print("Not Sur", sum(yp==0))
cm = confusion_matrix(y_test, yp)
cm

0.8324022346368715
Sur 65
Not Sur 114


/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


array([[97, 13],
       [17, 52]])

In [12]:
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# import numpy as np
# model = RandomForestClassifier()
# n_estimators = [int(x) for x in np.linspace(start=1, stop=200, num = 20)]
# max_features = ['auto', 'sqrt']
# max_depth = [int(x) for x in np.linspace(1, 50, num = 15)]
# max_depth.append(None)
# min_samples_split = [2, 3, 5, 10, 12, 13, 15,17, 20]
# min_samples_leaf = [1, 2, 4, 6,8,9, 10]
# bootstrap = [True, False]
# random_grid = {'n_estimators': n_estimators,
#                'max_features': max_features,
#                'max_depth': max_depth,
#                'min_samples_split': min_samples_split,
#                'min_samples_leaf': min_samples_leaf,
#                'bootstrap': bootstrap}

# rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, cv = 4)
# # rf_random = GridSearchCV(model, random_grid, cv=4)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
# rf_random.fit(X_train, y_train)
# print(rf_random.best_score_)
# rf_random.best_params_

In [13]:
# from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
# import numpy as np
# model = XGBClassifier()
# random_grid = {'eta': [x for x in np.linspace(start=0.01, stop=0.2, num = 4)],
#                'max_depth':range(1,10,1),
#                'min_child_weight':range(1,13,2),
#                'gamma':[i/10.0 for i in range(0,5)],
#              'subsample':[i/100.0 for i in range(60,100, 5)],
#              'colsample_bytree':[i/100.0 for i in range(60,100, 5)],
#               'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
#               }

# rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, cv = 4)
# # rf_random = GridSearchCV(model, random_grid, cv=5)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
# rf_random.fit(X_train, y_train)
# print(rf_random.best_score_)
# rf_random.best_params_

In [14]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np
model = XGBClassifier()
random_grid = {'eta': [x for x in np.linspace(start=0, stop=0.4, num = 10)]            }
# rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, cv = 4)
rf_random = GridSearchCV(model, random_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
rf_random.fit(X_train, y_train)
print(rf_random.best_score_)
rf_random.best_params_

/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

0.8075842696629213


/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

{'eta': 0.0}

In [15]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np
model = XGBClassifier(max_depth=2, min_child_weight= 3, gamma=0, 
                     subsample=0.86, reg_alpha=0)
random_grid = {'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]}
# rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, cv = 4)
rf_random = GridSearchCV(model, random_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
rf_random.fit(X_train, y_train)
print(rf_random.best_score_)
rf_random.best_params_

/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

0.8202247191011236


/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'reg_alpha': 0.005}

In [16]:
model  
#        max_depth=4, 
#        min_child_weight=9, 
#        n_estimators=10,
#       scale_pos_weight=0.9,
#       subsample=1


#        eta=0,  
#        max_depth=2, 
#         min_child_weight=3, 
#        n_estimators=100,
#        scale_pos_weight=1, 
#        subsample=0.86    
    

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=2, min_child_weight=3, missing=None,
       n_estimators=100, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=0.86, verbosity=1)

In [21]:
# random_grid = {'eta': [x for x in np.linspace(start=0.01, stop=0.2, num = 4)],
#                'max_depth':range(1,10,1),
#                'min_child_weight':range(1,13,2),
#                'gamma':[i/10.0 for i in range(0,5)],
#              'subsample':[i/100.0 for i in range(60,100, 5)],
#              'colsample_bytree':[i/100.0 for i in range(60,100, 5)],
#               'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
#               }

from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np
model = XGBClassifier(max_depth=2, min_child_weight= 3, gamma=0, 
                     subsample=0.86, reg_alpha=0)
random_grid = {'n_estimators':[int(x) for x in np.linspace(start=50, stop=200, num = 5)]}
# rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, cv = 4)
rf_random = GridSearchCV(model, random_grid, cv=5)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)
rf_random.fit(X_train, y_train)
print(rf_random.best_score_)
rf_random.best_params_

/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

0.826645264847512


/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'n_estimators': 125}

In [18]:
model = XGBClassifier(max_depth=2, min_child_weight= 3, gamma=0, 
                     subsample=0.86, reg_alpha=0, n_estimators=125)
model.fit(X, y)
print(model.score(X_test, y_test))

0.9162011173184358


/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [24]:
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import numpy as np
model = XGBClassifier()
random_grid = {'eta': [x for x in np.linspace(start=0.01, stop=0.2, num = 4)],
               'max_depth':range(1,10,1),
               'min_child_weight':range(1,13,2),
               'gamma':[i/10.0 for i in range(0,5)],
             'subsample':[i/100.0 for i in range(60,100, 5)],
             'colsample_bytree':[i/100.0 for i in range(60,100, 5)],
              'reg_alpha':[0, 0.001, 0.005, 0.01, 0.05]
              }
rf_random = RandomizedSearchCV(estimator = model, param_distributions = random_grid, cv = 4)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)
rf_random.fit(X_train, y_train)
print(rf_random.best_score_)
rf_random.best_params_

/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is a

0.8089887640449438


/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


{'colsample_bytree': 0.65,
 'eta': 0.07333333333333333,
 'gamma': 0.4,
 'max_depth': 3,
 'min_child_weight': 1,
 'reg_alpha': 0.05,
 'subsample': 0.6}

In [25]:
model = XGBClassifier(subsample= 0.85,
 reg_alpha = 0.01,
 min_child_weight = 5,
 max_depth = 3,
 gamma = 0.2,
 eta= 0.07333333333333333,
 colsample_bytree =  0.7, n_estimators=125)
model.fit(X, y)
print(model.score(X_test, y_test))

0.9106145251396648


/home/gaurav/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
